Este es un "notebook" (cuaderno digital) para usar el modelo de regresión logística para el modelo Ising.

Acompaña al Capítulo 5 del libro (4 de 5).

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman. Traducido por Manuel Pichardo Marcano y Genaro Suárez; ver también los créditos de los datos a continuación.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
from matplotlib import cm
%matplotlib inline

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split

from sklearn.model_selection import KFold, StratifiedKFold

from sklearn import metrics

### ¡Primero, echemos un vistazo a esos sigmoides!

In [ ]:
x = np.linspace(-10,10,100)

In [ ]:
z = 2*x + 5 #modelo lineal

Digamos que la probabilidad de que algo suceda se llama $\pi$.

El modelo logístico supone que

$log (\frac{\pi}{1-\pi}$) = z 

Ahora podemos resolver para $\pi$:

In [ ]:
pi = 1/(1 + np.exp(-z))

In [ ]:
plt.plot(x, pi)

plt.xlim(-7,3);

plt.title('¡Hola soy una sigmoide!')

plt.xlabel('x', fontsize=14)

plt.ylabel('$\pi$',fontsize=14);

### Registro de Aprendizaje
    
¿Dónde está $\pi$ = 0.5?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Mirando la definición del modelo logístico, podemos ver que $\pi$=0.5 (las probabilidades son las mismas) cuando z=0; en nuestro gráfico, esto corresponde a x=-2.5.
```
    
</p>
</details>

</br>

¿Qué sucede si la pendiente del modelo lineal es negativa?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Las asíntotas del sigmoide están invertidas y la curva disminuye monótonamente.
```
    
</p>
</details>

### Ahora podemos ver un ejemplo de Mehta et al 2018 (escrito en inglés):

["A high-bias, low-variance introduction to Machine Learning for physicists"](https://arxiv.org/abs/1803.08823).

Estamos tratando de usar un modelo de regresión logística para predecir si un material está en una fase ordenada o desordenada, según su configuración de espín. En una fase ordenada, los giros están alineados. La representación es una red 2D, por lo que nuestras características son los estados de espín de cada elemento de la red. El modelo físico, conocido como modelo de Ising, predice que la transición depende de la temperatura y se difumina (para una red de tamaño finito), alrededor de una temperatura crítica $T_c$.

Los datos de entrenamiento se componen de 160.000 simulaciones de Monte Carlo en un rango de temperaturas y sus etiquetas.

Las posibles aplicaciones de este formalismo involucran la predicción de la temperatura crítica para sistemas más complejos.

La lectura de los datos puede tardar un poco.

In [ ]:
#Tomado prestado con permiso de los "notebooks" (cuadernos digitales) mantenidos por P. Mehta.

######### CARGAR DATOS
# Los datos consisten en 16*10000 muestras tomadas en T=np.arange(0.25,4.0001,0.25):
nombre_archivo_datos = '../data/Ising2DFM_reSample_L40_T=All.pkl'
# Las etiquetas se obtienen del siguiente archivo:
nombre_archivo_etiqueta = '../data/Ising2DFM_reSample_L40_T=All_labels.pkl'


#DATOS
with open(nombre_archivo_datos, 'rb') as pickle_file:
    datos = pickle.load(pickle_file) # pickle lee el archivo y devuelve el objeto Python (matriz 1D, bits comprimidos)


datos = np.unpackbits(datos).reshape(-1, 1600) # Descomprimir el arreglo y restructurarlo para mayor comodidad
datos = datos.astype('int')
datos[np.where(datos==0)]=-1 # cambiar valores 0 al estado -1 (la variable Ising puede tomar valores +/-1)

#ETIQUETAS (la convención es 1 para estados ordenados y 0 para estados desordenados)
with open(nombre_archivo_etiqueta, 'rb') as pickle_file:
    etiquetas = pickle.load(pickle_file) # pickle lee el archivo y devuelve el objeto de Python (aquí solo una matriz 1D con las etiquetas binarias)


In [ ]:
datos.shape

In [ ]:
np.unique(etiquetas)

Podemos echar un vistazo a la distribución de etiquetas:

In [ ]:
plt.scatter(np.arange(datos.shape[0]),etiquetas)

plt.xlabel('# de Ejemplo')

plt.ylabel('Estado');

#etiquetas: 1 = ordenada o casi crítica
#etiquetas: 0 = desordenado

### Registro de Aprendizaje
    
¿Es este un conjunto de datos equilibrado o desequilibrado?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Para verificar el balance, podemos contar el porcentaje de etiquetas "1", p. haciendo np.sum(etiquetas)/len(etiquetas), y obtenemos ~56%, lo que indica que el conjunto de datos está balanceado.
```
    
</p>
</details>

#### Podemos echar un vistazo a algunos ejemplos:

In [ ]:
#H/T: https://stackoverflow.com/questions/16834861/create-own-colormap-using-matplotlib-and-plot-color-scale

cmap = matplotlib.colors.ListedColormap(["aquamarine","navy"], name='from_list', N=None)

plt.figure(figsize=(15,8))
fig, axarr = plt.subplots(nrows=1, ncols=3)
axarr[0].imshow(datos[0].reshape(40,40), cmap = cmap) #el primer objeto tiene la etiqueta "1"
axarr[1].imshow(datos[80000].reshape(40,40), cmap = cmap) #de la documentación, esto es crítico (entre 60 y 90,000)
axarr[2].imshow(datos[100000].reshape(40,40), cmap = cmap) #desordenado
for i in range(3):
    axarr[i].set_xticks([0,20,40]);

### Elijamos una selección aleatoria para acelerar los cálculos.

In [ ]:
np.random.seed(10)

sel = np.random.choice(datos.shape[0], 16000, replace = False)

In [ ]:
seldatos = datos[sel,:]

In [ ]:
seletiquetas = etiquetas[sel]

In [ ]:
plt.scatter(np.arange(seldatos.shape[0]),seletiquetas); #¡La selección aleatoria también tiene la ventaja de reorganizar los datos!

### Y ahora es el momento del modelo de regresión logística.

In [ ]:
modelo = LogisticRegression(max_iter = 1000) #Esto utiliza un método numérico para encontrar el mínimo de la función de pérdida

In [ ]:
modelo.get_params() #Notar que (a diferencia de la regresión lineal) ¡la regularización es la norma!

In [ ]:
modelo

Podemos usar la validación cruzada, como de costumbre:

In [ ]:
#Toma 5-10 segundos

results = cross_validate(modelo, seldatos, seletiquetas, 
                         cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score = True)

In [ ]:
results 

### Registro de Aprendizaje
    
¿Qué métrica crees que representan esos números?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Sorprendentemente, el resultado estándar de Regresión Logística es la precisión (¡una métrica de clasificación!)
```

</p>

</details>

Este comportamiento es subóptimo porque también queremos acceder a las probabilidades. Veremos eso en un momento.

### Podemos hacer nuestra propia búsqueda en cuadrícula para optimizar el parámetro de regularización C:

In [ ]:
for C in np.logspace(-3,3,7):
    modelo = LogisticRegression(max_iter=1000, C = C)
    resultados = cross_validate(modelo, seldatos, seletiquetas, 
                         cv = KFold(n_splits=5, shuffle=True, random_state=10), return_train_score = True)
    print('C/Precisión promedio de prueba para C = ', '{:.3e} {:s} {:.3f} {:s} {:.3f}'.format(C, 'is ', results['test_score'].mean(),'+-',results['test_score'].std()))
    print('C/Precisión promedio del entrenamiento para C = ', '{:.3e} {:s} {:.3f} {:s} {:.3f}'.format(C, 'is ', results['train_score'].mean(),'+-',results['train_score'].std()))

### Preguntas:

- ¿Cómo es el rendimiento de este modelo?


### Registro de Aprendizaje

¿Qué valor de C debemos elegir?

<br>

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Las notas (de la prueba) son bastante planos, por lo que el valor de C que elijamos no es tan importante.
```

</p>
</details>

</br>

¿Cómo es el rendimiento de este modelo?

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
No muy bien, con una precisión de alrededor del 66%.
```

</p>
</details>
    

### Aquí generamos etiquetas para comprobar las predicciones.

Para aquellos clasificadores que están resolviendo un problema de regresión difícil de resolver, existe el método práctico "predict_proba".

In [ ]:
modelo = LogisticRegression(C=1.0, max_iter=1000)

ypred = cross_val_predict(modelo, seldatos, seletiquetas, \
                               cv = KFold(n_splits=5, shuffle=True, random_state=10))

ypred_prob = cross_val_predict(modelo, seldatos, seletiquetas, \
                               cv = KFold(n_splits=5, shuffle=True, random_state=10), method = 'predict_proba')

La salida de predict_proba da la probabilidad de pertenecer a la fase desordenada (etiqueta 0) u ordenada (etiqueta 1). La salida del clasificador simple es la clase con p > 0.5. Podemos mirar esto para convencernos:

In [ ]:
np.column_stack([ypred_prob, ypred])

### Podemos graficar algunos ejemplos para ver cómo está funcionando nuestro clasificador.

In [ ]:
fig, axarr = plt.subplots(nrows=1, ncols=8, figsize=(15,5))
for i in range(8):
    axarr[i].imshow(seldata[i].reshape(40,40), cmap = cmap) 
    axarr[i].set_xlabel('Etiqueta verdadera:'+str(sellabels[i])+'\n'+'Etiqueta pred:'+str(ypred[i]))
    axarr[i].set_yticks([])
    axarr[i].set_xticks([])

Desafortunadamente, hay dos instancias que están mal clasificadas por nuestro clasificador Log Reg. Sin embargo, al menos visualmente, ¡es comprensible!

Sin embargo, una mirada a las probabilidades correspondientes revela algunas preocupaciones:

In [ ]:
ypred_prob[:8]

0 Ordenado (confianza decente)

1 Ordenado (confianza decente)

Se predice que 2 se ordenará CON ALTA CONFIANZA... ¡PERO INCORRECTAMENTE!

.....

Algo anda mal aquí, porque el nivel de confianza de los casos muy inciertos parece ser demasiado alto.

La conclusión es que el indicador principal de este modelo es la falta de consistencia entre las alineaciones de espín, que no está bien modelada por nuestro regresor. Es un problema complicado porque muchos algoritmos tienden a mirar el valor de cada función para decidir; para muchos de ellos, es difícil representar la correlación entre las funciones como un indicador.

### Registro de Aprendizaje

¿Qué algoritmo de los que hemos visto hasta ahora recomendaría usar en lugar de Regresión logística?

<br>

<details>
<summary style="display: list-item;">¡Haz clic aquí para la respuesta!</summary>
<p>
    
```
Algo que parece importante aquí es poder combinar funciones entre sí. Esto es algo que los modelos lineales (generalizados) no pueden hacer bien, pero está al alcance del algoritmo de vectores de soporte, por ejemplo.
```
    
</p>
</details>
